In [2]:
import gzip
import pickle
import networkx as nx
import pandas as pd
import numpy as np
import random
import os

In [3]:
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau

/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Loading

### Writing PyG Dataset

In [4]:
import torch

In [5]:
from torch_geometric.data import InMemoryDataset
from torch.utils.data import DataLoader

In [6]:
from torch_geometric.utils import from_networkx, to_networkx

In [7]:
def clustering_coefficient(G, node):
    ns = [n for n in G.neighbors(node)]
    if len(ns) <= 1:
        return 0
    
    numerator = 0
    denominator = len(ns) * (len(ns) - 1) / 2
    for i in range(0, len(ns)):
        for j in range(i+1, len(ns)):
            n1, n2 = ns[i], ns[j]
            numerator += G.has_edge(n1, n2)
    
    return numerator / denominator
                

In [8]:
def generate_feature_vector(G):
    x = torch.randn(G.number_of_nodes(), 2)
    ind = 0
    for node in G.nodes():
        # set node degree as feature
        x[ind][0] = G.degree[node]
        x[ind][1] = clustering_coefficient(G, node)
        ind += 1
    return x

In [9]:
class LamanDataset(InMemoryDataset):
    def __init__(self, root, data_dir, transform=None, pre_transform=None, pre_filter=None):
        self.data_dir = data_dir
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data.pt']
        
    def process(self):
        total_laman_data = None
        with gzip.open(self.data_dir, 'r') as f:
            total_laman_data = pickle.load(f)
            
        data_list = []
        for ind, graph in enumerate(total_laman_data[0]):
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 0
            data_list.append(graph_as_data)
            
        for ind, graph in enumerate(total_laman_data[1]):
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 1
            data_list.append(graph_as_data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [10]:
DATA_PATH = "../data-2d/data/4096-20-4-entries-med.pkl.gz"

In [11]:
laman_data = LamanDataset("", DATA_PATH)

In [12]:
laman_data[0]

Data(edge_index=[2, 122], x=[32, 2], label=[1], num_nodes=32)

## Split into Train / Test

In [13]:
from torch.utils.data import random_split

proportions = [.6, .4]
lengths = [int(p * len(laman_data)) for p in proportions]
lengths[-1] = len(laman_data) - sum(lengths[:-1])

generator1 = torch.Generator().manual_seed(42)
train_data, test_data = random_split(laman_data, lengths, generator=generator1)

In [14]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_data, batch_size = 256, shuffle=True)
test_loader = DataLoader(test_data, batch_size = 256, shuffle=True)

In [15]:
print("Number of train batches: ", len(train_loader))
print("Number of test batches: ", len(test_loader))

Number of train batches:  10
Number of test batches:  7


In [16]:
for data in train_loader:
    print(data)
    break

DataBatch(edge_index=[2, 18776], x=[5034, 2], label=[256], num_nodes=5034, batch=[5034], ptr=[257])


## Model Architecture & Training

In [17]:
from gin.gin import GIN

In [18]:
model = GIN(num_features=2)
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

GIN(
  (conv1): GINConv(nn=Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=2, out_features=2, bias=True)
    (4): ReLU()
  ))
  (conv2): GINConv(nn=Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=2, out_features=2, bias=True)
    (4): ReLU()
  ))
  (conv3): GINConv(nn=Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=2, out_features=2, bias=True)
    (4): ReLU()
  ))
  (conv4): GINConv(nn=Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [19]:
from torch_geometric.data import DataLoader
from torch.nn import BCELoss
import warnings
warnings.filterwarnings("ignore")

loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),
                                      lr=0.001)

# scheduler = ReduceLROnPlateau(optimizer, 'min', min_lr=1e-6, verbose=True, patience=10)

In [20]:
def train(model, data, features_to_use):
    ind = 0
    for batch in data:
        optimizer.zero_grad()
        pred, embedding = model(batch.x[:, features_to_use], batch.edge_index, batch.batch)
        pred = torch.squeeze(pred)
        loss = loss_fn(pred.float(), batch.label.float())
        loss.backward()
        optimizer.step()
        ind += 1

    return loss, model

In [21]:
def check_accuracy(model, loader, features_to_use):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for batch in loader:
            pred, embedding = model(batch.x[:, features_to_use], batch.edge_index, batch.batch)
            pred = torch.squeeze(pred)
            y = batch.label
            predictions = (pred > 0.5).long() 
            num_correct += (predictions == y).sum() 
            num_samples += predictions.size(0)
            
    return float(num_correct)/float(num_samples)*100

In [22]:
print("Starting training...")
losses = []

bestModel, highestAcc = None, 0

for epoch in range(1000):
    loss, model = train(model, train_loader, [0, 1])
    losses.append(loss)
    print(f"Epoch {epoch} | Train loss {loss}")
    train_acc, test_acc = check_accuracy(model, train_loader, [0, 1]), check_accuracy(model, test_loader, [0, 1])
    print(f"Train Accuracy {train_acc} | Test Accuracy {test_acc}")
#     scheduler.step(test_acc)
    
    if test_acc > highestAcc:
        highestAcc = test_acc
        bestModel = model

Starting training...
Epoch 0 | Train loss 0.6044948101043701
Train Accuracy 65.77126577126577 | Test Accuracy 65.95485051860891
Epoch 1 | Train loss 0.6098939180374146
Train Accuracy 65.77126577126577 | Test Accuracy 65.95485051860891
Epoch 2 | Train loss 0.6249003410339355
Train Accuracy 65.77126577126577 | Test Accuracy 65.95485051860891
Epoch 3 | Train loss 0.6085838675498962
Train Accuracy 65.77126577126577 | Test Accuracy 65.95485051860891
Epoch 4 | Train loss 0.6361799240112305
Train Accuracy 65.77126577126577 | Test Accuracy 65.95485051860891
Epoch 5 | Train loss 0.618030846118927
Train Accuracy 65.8933658933659 | Test Accuracy 66.01586333129957
Epoch 6 | Train loss 0.5934194922447205
Train Accuracy 66.0968660968661 | Test Accuracy 65.95485051860891
Epoch 7 | Train loss 0.5735112428665161
Train Accuracy 66.95156695156696 | Test Accuracy 67.41915802318486
Epoch 8 | Train loss 0.6135768294334412
Train Accuracy 67.43996743996745 | Test Accuracy 68.33435021354485
Epoch 9 | Train los

Epoch 76 | Train loss 0.18575449287891388
Train Accuracy 94.0984940984941 | Test Accuracy 93.95973154362416
Epoch 77 | Train loss 0.19238246977329254
Train Accuracy 93.48799348799349 | Test Accuracy 93.34960341671751
Epoch 78 | Train loss 0.2147490531206131
Train Accuracy 94.46479446479447 | Test Accuracy 94.14276998169616
Epoch 79 | Train loss 0.17558468878269196
Train Accuracy 92.38909238909238 | Test Accuracy 92.67846247712019
Epoch 80 | Train loss 0.1756524294614792
Train Accuracy 93.52869352869352 | Test Accuracy 93.47162904209884
Epoch 81 | Train loss 0.1841973513364792
Train Accuracy 94.46479446479447 | Test Accuracy 94.08175716900548
Epoch 82 | Train loss 0.1860804259777069
Train Accuracy 94.26129426129425 | Test Accuracy 94.5698596705308
Epoch 83 | Train loss 0.17216694355010986
Train Accuracy 95.23809523809523 | Test Accuracy 94.69188529591214
Epoch 84 | Train loss 0.15657725930213928
Train Accuracy 94.42409442409443 | Test Accuracy 94.26479560707749
Epoch 85 | Train loss 0.2

Train Accuracy 98.61619861619862 | Test Accuracy 98.7797437461867
Epoch 152 | Train loss 0.0882212221622467
Train Accuracy 98.9010989010989 | Test Accuracy 99.08480780964003
Epoch 153 | Train loss 0.07309981435537338
Train Accuracy 98.2091982091982 | Test Accuracy 98.23062843197071
Epoch 154 | Train loss 0.06353852897882462
Train Accuracy 98.94179894179894 | Test Accuracy 99.20683343502135
Epoch 155 | Train loss 0.0583437941968441
Train Accuracy 99.3080993080993 | Test Accuracy 98.84075655887736
Epoch 156 | Train loss 0.05433236062526703
Train Accuracy 98.98249898249898 | Test Accuracy 99.08480780964003
Epoch 157 | Train loss 0.07300922274589539
Train Accuracy 98.53479853479854 | Test Accuracy 98.65771812080537
Epoch 158 | Train loss 0.08992498368024826
Train Accuracy 98.4940984940985 | Test Accuracy 98.53569249542404
Epoch 159 | Train loss 0.10562876611948013
Train Accuracy 99.14529914529915 | Test Accuracy 98.59670530811471
Epoch 160 | Train loss 0.06627199053764343
Train Accuracy 98

Epoch 226 | Train loss 0.021558355540037155
Train Accuracy 99.83719983719985 | Test Accuracy 99.45088468578402
Epoch 227 | Train loss 0.03497352451086044
Train Accuracy 99.83719983719985 | Test Accuracy 99.51189749847468
Epoch 228 | Train loss 0.029473721981048584
Train Accuracy 99.63369963369964 | Test Accuracy 99.51189749847468
Epoch 229 | Train loss 0.026982827112078667
Train Accuracy 99.34879934879935 | Test Accuracy 99.38987187309336
Epoch 230 | Train loss 0.03535410016775131
Train Accuracy 99.83719983719985 | Test Accuracy 99.57291031116534
Epoch 231 | Train loss 0.0234312042593956
Train Accuracy 99.83719983719985 | Test Accuracy 99.57291031116534
Epoch 232 | Train loss 0.026615804061293602
Train Accuracy 99.87789987789988 | Test Accuracy 99.51189749847468
Epoch 233 | Train loss 0.05049692839384079
Train Accuracy 99.87789987789988 | Test Accuracy 99.63392312385601
Epoch 234 | Train loss 0.022268274798989296
Train Accuracy 99.1045991045991 | Test Accuracy 99.02379499694935
Epoch 2

KeyboardInterrupt: 

In [23]:
# sanity check

In [24]:
# test on best model

In [25]:
# square
import networkx as nx
square = nx.Graph()
square.add_edge(0, 1)
square.add_edge(1, 3)
square.add_edge(0, 2)
square.add_edge(2, 3)

In [26]:
# square with cross bar (rigid)
import networkx as nx
square_bar = nx.Graph()
square_bar.add_edge(0, 1)
square_bar.add_edge(1, 3)
square_bar.add_edge(0, 2)
square_bar.add_edge(2, 3)
square_bar.add_edge(0, 3)

In [27]:
# triangle
import networkx as nx
triangle = nx.Graph()
triangle.add_edge(0, 1)
triangle.add_edge(0, 2)
triangle.add_edge(1, 2)

In [28]:
# pentagon
import networkx as nx
pentagon = nx.Graph()
pentagon.add_edge(0, 1)
pentagon.add_edge(1, 3)
pentagon.add_edge(3, 4)
pentagon.add_edge(4, 2)
pentagon.add_edge(2, 0)

In [29]:
# no triangle and rigid
rigid = nx.Graph()
rigid.add_edge(0, 1)
rigid.add_edge(0, 2)
rigid.add_edge(0, 4)
rigid.add_edge(1, 2)
rigid.add_edge(1, 5)
rigid.add_edge(2, 3)
rigid.add_edge(3, 4)
rigid.add_edge(3, 5)
rigid.add_edge(4, 5)

In [30]:
toy_problems = [square, square_bar, triangle, pentagon, rigid]
labels = [1, 0, 0, 1, 0]

for index, toy_problem in enumerate(toy_problems):
    graph_as_data = from_networkx(toy_problem)
    graph_as_data.x = generate_feature_vector(toy_problem)
    graph_as_data.label = labels[index]
    validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)
    for batch in validation_set:
        pred = model(batch.x[:, [0, 1]], batch.edge_index, batch.batch)
        print(pred[0])
    

tensor([[0.9944]], grad_fn=<SigmoidBackward0>)
tensor([[0.0513]], grad_fn=<SigmoidBackward0>)
tensor([[0.6229]], grad_fn=<SigmoidBackward0>)
tensor([[0.9998]], grad_fn=<SigmoidBackward0>)
tensor([[0.0747]], grad_fn=<SigmoidBackward0>)


In [31]:
graph_as_data

Data(edge_index=[2, 18], num_nodes=6, x=[6, 2], label=0)

In [ ]:
validation_set = DataLoader([graph_as_data], batch_size = 1, shuffle=True)

In [ ]:
for batch in validation_set:
    pred = bestModel(batch.x, batch.edge_index, batch.batch)
    print(pred[0])
    print(pred[1])

In [32]:
class LamanTestDataset(InMemoryDataset):
    def __init__(self, root, data_dir, transform=None, pre_transform=None, pre_filter=None):
        self.data_dir = data_dir
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def processed_file_names(self):
        return ['data_test.pt']
        
    def process(self):
        # processing code here
        total_laman_data = None
        with gzip.open(self.data_dir, 'r') as f:
            total_laman_data = pickle.load(f)
            
        data_list = []
        ind = 0
        # convert from graph to Data object
        for graph in total_laman_data[0]:
#             print(ind)
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
#             x = torch.randn(num_nodes, 1)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 0
            data_list.append(graph_as_data)
            
        ind = 0
        for graph in total_laman_data[1]:
#             print(ind)
            ind += 1
            num_nodes = nx.number_of_nodes(graph)
#             x = torch.randn(num_nodes, 64)
            x = generate_feature_vector(graph)
            graph_as_data = from_networkx(graph)
            graph_as_data.x = x
            graph_as_data.label = 1
            data_list.append(graph_as_data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [33]:
# add functionality to support a test dataset
TEST_DATA_PATH = "../data-2d/data/test-dataset-30loc-5std.pkl.gz"
laman_test_set = LamanTestDataset("", TEST_DATA_PATH)

Processing...
Done!


In [34]:
from torch_geometric.loader import DataLoader
laman_test_loader = DataLoader(laman_test_set, batch_size = 2, shuffle=True)

In [35]:
random_test_acc = check_accuracy(model, laman_test_loader, [0, 1])
print(f"Accuracy {random_test_acc}")

Accuracy 96.09375


In [ ]:
# generate statistics on the data

In [ ]:
# test the clustering coefficient

In [ ]:
clustering_coefficient(square, 0)

In [ ]:
clustering_coefficient(triangle, 0)

In [ ]:
clustering_coefficient(square_bar, 0)

In [ ]:
# generate graph correlating clustering coefficient to rigidity

In [ ]:
for item in train_data:
    item = to_networkx(item)
    print(type(item))
    
    break

In [ ]:
torch_geometric.utils.convert.to_networkx()

# Scratch Work: Sahil

In [ ]:
total_laman_data = None
with gzip.open(DATA_PATH, 'r') as f:
    total_laman_data = pickle.load(f)

In [ ]:
sample_graph = total_laman_data[0][0]

In [ ]:
print(type(sample_graph))

In [ ]:
to_data = from_networkx(sample_graph)
from_data = to_networkx(to_data, to_undirected = True)

In [ ]:
print(type(from_data))

In [ ]:
def compute_min_clustering_coefficient(G):
    min_coefficient = 1
    for node in G.nodes():
        min_coefficient = min(min_coefficient, clustering_coefficient(G, node))
        
    return min_coefficient

In [ ]:
for index, sample_graph in enumerate(train_data):
    label = sample_graph.label
    networkx_sample_graph = to_networkx(sample_graph, to_undirected = True)
    print(label, " ", index, " ", compute_min_clustering_coefficient(networkx_sample_graph))
    
    if index == 10:
        break

In [ ]:
print(compute_min_clustering_coefficient(from_data))

In [ ]:
compute_min_clustering_coefficient(triangle)

In [ ]:
# what if instead of training a gnn – you just trained on 

In [ ]:
# train a binary classifier on just the degrees of the nodes

In [ ]:
# train a binary classifier on just the triangle feature

In [ ]:
# train a binary classifier on the triangle features and the degree

In [ ]:
# train a network with just degree of the node 

In [ ]:
model_just_degree = GIN(num_features=1)
print(model_just_degree)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

In [ ]:
print("Starting training...")
losses = []

bestModel, highestAcc = None, 0

for epoch in range(1000):
    loss, h = train(train_loader, [0, 1])
    losses.append(loss)
    print(f"Epoch {epoch} | Train loss {loss}")
    train_acc, test_acc = check_accuracy(model_just_degree, train_loader), check_accuracy(model_just_degree, test_loader)
    print(f"Train Accuracy {train_acc} | Test Accuracy {test_acc}")
#     scheduler.step(test_acc)
    
    if test_acc > highestAcc:
        highestAcc = test_acc
        bestModel = model

In [ ]:
generate_feature_vector(sample_graph)

In [ ]:
sample_graph.x[:, [0, 1]]